#  normalize NOAA DSDP taxa list
## 1-96 taxa

Cleanup the normalized taxa list from the eODP researchers.

In [3]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [4]:
# date = '2021-07-28'
# date = '2021-08-05'
# date='2021-11-29'
# date = '2022-09-12'
# date='2022-10-26'
date='2022-11-15'
date='2023-10-11'

# prev_date = '2022-09-12'
# prev_date = '2022-11-15'



base_dir = CLEAN_DATA_DIR

initial_input_file = RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
input_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"
crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_crosswalk_{date}.csv"
taxa_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_list_{date}.csv"

genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_pbdb_{date}.csv'
higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_pbdb_{date}.csv'  

taxa_pbdb_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_list_pbdb_{date}.csv'

# old_genus_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'genus_pbdb_{prev_date}.csv'
# old_higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'higher_taxa_pbdb_{prev_date}.csv'  



## fix incorect pbdb_taxon_id

incorported  pbdb_taxon_id that the PIs corrected into the taxalist.


In [46]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)
# (7763, 39)

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [47]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'Corrections to pbdb_taxon_rank',
       'pbdb_taxon_id.1', 'pbdb_taxon_name.1', 'pbdb_taxon_rank.1',
       'Corrections to pbdb_taxon', 'family_taxon_id', 'family_taxon_name',
       'superfamily_taxon_id', 'superfamily_taxon_name', 'order_taxon_id',
       'order_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [48]:
col = 'pbdb_taxon_id'
PI_df[PI_df[col] !=PI_df[col+'.1']][['verbatim_name', col, col+'.1']]

,verbatim_name,pbdb_taxon_id,pbdb_taxon_id.1
12,Alabinella wedellensis,NaN,423746
15,Alveolophraginium crassimargo,NaN,805
95,Aragonina aragonensis,NaN,865
96,Aragonina velascoensis,NaN,865
112,Astrohiza granulosa,NaN,912
...,...,...,...
7747,Pseudomicromarsupium lacrimosa,NaN,NaN
7753,Septamesocena apiculata,NaN,NaN
7754,Septamesocena quadrangula (q),NaN,NaN
7755,Septamesocena sp.,NaN,NaN


In [49]:
if 'pbdb_taxon_id.1' in PI_df:

    PI_df = PI_df.rename(columns={ 
        'pbdb_taxon_id': 'pbdb_taxon_id_old',
        'pbdb_taxon_name': 'pbdb_taxon_name_old',
        'pbdb_taxon_rank': 'pbdb_taxon_rank_old',
        'pbdb_taxon_id.1': 'pbdb_taxon_id',
        'pbdb_taxon_name.1': 'pbdb_taxon_name',
        'pbdb_taxon_rank.1': 'pbdb_taxon_rank'
    })

In [50]:
cols = [
    'Corrections to pbdb_taxon_rank',
    'Corrections to pbdb_taxon',
    'pbdb_taxon_id_old',
    'pbdb_taxon_id'
]
tmp = PI_df[PI_df['Corrections to pbdb_taxon_rank'].notna() | 
            PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon_rank,Corrections to pbdb_taxon,pbdb_taxon_id_old,pbdb_taxon_id
295,NaN,1059,383509,383509
299,genus; id 1064,1064,168467,1064
438,"genus, id 1124",NaN,70260,1124
1169,genus taxon_no=2092,2092,153914,2092
1170,genus taxon_no=2092,2092,153914,153914
1411,genus taxon_no=2542,NaN,137071,2542
1640,NaN,441243,374615,374615
1752,genus; ID 71247,NaN,9512,71247
1989,genus; ID 82145,NaN,14728,82145
2143,Note: homonym with a plant once this is entere...,NaN,319949,319949


In [51]:
correct_col = 'Corrections to pbdb_taxon_rank'
pbdb.fix_pbdb_id(PI_df, 'genus; id 1064', 1064, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus, id 1124', 1124, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus taxon_no=2092', 2092, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus taxon_no=2542', 2542, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 71247', 71247, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 82145', 82145, correct_col)
# pbdb.fix_pbdb_id(PI_df, 'Note: homonym with a plant once this is entered, update taxon #', None, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 432650', 432650, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus: ID 68421', 68421, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 432651', 432651, correct_col)
pbdb.fix_pbdb_id(PI_df, 'homonym with an isect; ID# is 414258', 414258, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 421517', 421517, correct_col)
# pbdb.fix_pbdb_id(PI_df, 'homonym with a plant', None, correct_col)
# pbdb.fix_pbdb_id(PI_df, 'homonym with heart urchin', None, correct_col)
pbdb.fix_pbdb_id(PI_df, 'ID# 24521', 24521, correct_col)
pbdb.fix_pbdb_id(PI_df, '455011', 455011, correct_col)
pbdb.fix_pbdb_id(PI_df, '455012', 455012, correct_col)
pbdb.fix_pbdb_id(PI_df, '30991', 30991, correct_col)
pbdb.fix_pbdb_id(PI_df, 'genus; ID 165526', 165526, correct_col)
pbdb.fix_pbdb_id(PI_df, '454776', 454776, correct_col)



correct_col = 'Corrections to pbdb_taxon'
pbdb.fix_pbdb_id(PI_df, '1059', 1059, correct_col)
pbdb.fix_pbdb_id(PI_df, '2092', 2092, correct_col)
pbdb.fix_pbdb_id(PI_df, '441243', 441243, correct_col)
pbdb.fix_pbdb_id(PI_df, '441540', 441540, correct_col)
pbdb.fix_pbdb_id(PI_df, '445341', 445341, correct_col)
pbdb.fix_pbdb_id(PI_df, '414258', 414258, correct_col)
pbdb.fix_pbdb_id(PI_df, '454352', 454352, correct_col)
pbdb.fix_pbdb_id(PI_df, '53873', 53873, correct_col)
pbdb.fix_pbdb_id(PI_df, '117902', 117902, correct_col)
pbdb.fix_pbdb_id(PI_df, '84824', 84824, correct_col)
pbdb.fix_pbdb_id(PI_df, '252', 252, correct_col)
pbdb.fix_pbdb_id(PI_df, '454594', 454594, correct_col)


correct_col = 'Corrections to pbdb_taxon'
pbdb.fix_pbdb_id(PI_df, '453602', 453602, correct_col)
pbdb.fix_pbdb_id(PI_df, '444642', 444642, correct_col)
pbdb.fix_pbdb_id(PI_df, '84824', 84824, correct_col)
pbdb.fix_pbdb_id(PI_df, '252', 252, correct_col)

correct_col = 'Corrections to pbdb_taxon_rank'
pbdb.fix_pbdb_id(PI_df, '455011', 455011, correct_col)
pbdb.fix_pbdb_id(PI_df, '455012', 455012, correct_col)
pbdb.fix_pbdb_id(PI_df, '30991', 30991, correct_col)
pbdb.fix_pbdb_id(PI_df, '455013', 455013, correct_col)

 


1064
1124
2092
2542
71247
82145
432650
68421
432651
414258
421517
24521
455011
455012
30991
165526
454776
1059
2092
441243
441540
445341
414258
454352
53873
117902
84824
252
454594
453602
444642
84824
252
455011
455012
30991
455013


In [53]:
del PI_df['corrected']

In [54]:
PI_df.shape
# 7763

(7763, 39)

In [55]:
# PI_df = PI_df.reindex(columns=[
#     'taxon_group', 'verbatim_name', 
#     'name', 'Comment',
#     'Notes (change to Internal only notes?)',
#     'Any taxon above genus modifier', 'Any taxon above genus',
#     'genus modifier', 'genus name', 
#     'subgenera modifier', 'subgenera name',
#     'species modifier', 'species name', 
#     'subspecies modifier','subspecies name', 
#     'non-taxa descriptor', 
#     'comments',
#     'Corrections to pbdb_taxon_rank', 
#     'pbdb_taxon_id_old', 'pbdb_taxon_name_old', 'pbdb_taxon_rank_old',
#     'pbdb_taxon_id', 'pbdb_taxon_name','pbdb_taxon_rank', 
#     'Corrections to pbdb_taxon', 
#     'family_taxon_id', 'family_taxon_name', 
#     'superfamily_taxon_id', 'superfamily_taxon_name', 
#     'order_taxon_id', 'order_taxon_name', 
#     'class_taxon_id', 'class_taxon_name', 
#     'phylum_taxon_id', 'phylum_taxon_name',
#     'kingdom_taxon_id', 'kingdom_taxon_name', 
#     'unranked clade_taxon_id','unranked clade_taxon_name'
# ])
# PI_df.shape

In [56]:
PI_df.to_csv(input_file, index=False)


## create genera csv

In [27]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [28]:
genus_df =  pbdb.create_genus_df(df)

log_df(genus_df)

(1604, 15)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,benthic_forams,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,benthic_forams,Adercotryma,774,Adercotryma,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,benthic_forams,Alabamina,788,Alabamina,genus,82213,Alabaminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
11,benthic_forams,Alabaminoides,789,Alabaminoides,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
12,benthic_forams,Alabaminella,423746,Alabaminella,genus,241423,Eponididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


In [23]:
genus_df[genus_df['pbdb_taxon_id'].isna()].shape

(244, 15)

In [29]:
genus_df[genus_df['genus name']=='Bulimina']

,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
134,benthic_forams,Bulimina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,benthic_forams,Bulimina,1032,Bulimina,genus,103766,Buliminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


check for duplicate genus

In [26]:
genus_df[genus_df.duplicated(subset=['taxon_group', 'genus name'])]

,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
243,benthic_forams,Bulimina,1032,Bulimina,genus,103766,Buliminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1019,benthic_forams,Nodosaria,1952,Nodosaria,genus,82197,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria
2080,diatoms,Denticulopsis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,diatoms,Rhabdonema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3159,dinoflagellates,Nematosphaeropsis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3297,ebridians,Ebriopsis,450955,Ebriopsis,genus,450956,Ebriopsidaceae,450954,Ebriales,NaN,NaN,NaN,NaN,NaN,NaN
3482,nannofossils,Crenalithus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3772,nannofossils,Fasciculithus,424283,Fasciculithus,genus,422531,Fasciculithaceae,422508,Discoasterales,418920,Coccolithophyceae,87644,Haptophyta,NaN,NaN
4106,nannofossils,Syracosphaera,270678,Syracosphaera,genus,NaN,NaN,NaN,NaN,NaN,NaN,87644,Haptophyta,NaN,NaN
4816,planktic_forams,Globigerinopsis,1507,Globigerinopsis,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


check for genus name used in more than taxon group

In [30]:
genus_df[genus_df.duplicated(subset=['genus name'])]

,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
243,benthic_forams,Bulimina,1032,Bulimina,genus,103766,Buliminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1019,benthic_forams,Nodosaria,1952,Nodosaria,genus,82197,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria
2080,diatoms,Denticulopsis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,diatoms,Rhabdonema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3159,dinoflagellates,Nematosphaeropsis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3204,dinoflagellates,Pseudorocella,413527,Pseudorocella,genus,NaN,NaN,NaN,NaN,69590,Chrysophyceae,NaN,NaN,NaN,NaN
3288,ebridians,Cinctactiniscus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3297,ebridians,Ebriopsis,450955,Ebriopsis,genus,450956,Ebriopsidaceae,450954,Ebriales,NaN,NaN,NaN,NaN,NaN,NaN
3482,nannofossils,Crenalithus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3772,nannofossils,Fasciculithus,424283,Fasciculithus,genus,422531,Fasciculithaceae,422508,Discoasterales,418920,Coccolithophyceae,87644,Haptophyta,NaN,NaN


In [32]:
pbdb.fetch_pdbd_data(genus_df, 'genus name')


0 50 Bulava  not found
450 "Discocibicides"  not found
1000 1200 Remanella  not found
1400 1800 Amiculosphaera  not found
2950 3200 Coronocyclas  not found
3850 Syracospahera  not found
4250 Paranesdiea  not found
4350 Gtheca  not found
Pseudogloboquadrina  not found
5600 5700 Gnetum  not found
"Gomphrenia"  not found
"Liguliflores"  not found
5850 Schizandra  not found
Schizoporis  not found
Tetrapollis  not found
5950 Tricopopollenites  not found
Tubiflores  not found
6050 Astostrobus  not found
6100 Clistophaera  not found
6350 6500 Lophochytris  not found
6650 6750 Porospathis  not found
Pterocyoticlius  not found
Pterycorys  not found
Siphostichoartus  not found
Spongolagena  not found
Trissospyris  not found
Xiphospira  not found


In [33]:
log_df(genus_df)

(1604, 19)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name
0,benthic_forams,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma,774,Adercotryma,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina,788,Alabamina,genus,82213,Alabaminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
11,benthic_forams,Alabaminoides,789,Alabaminoides,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
12,benthic_forams,Alabaminella,423746,Alabaminella,genus,241423,Eponididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN


In [34]:
genus_df[genus_df['pbdb_taxon_id'].isna()].shape

(27, 19)

In [41]:
genus_df.drop_duplicates(inplace=True)

log_df(genus_df)

(1593, 19)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name
0,benthic_forams,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma,774,Adercotryma,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina,788,Alabamina,genus,82213,Alabaminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
11,benthic_forams,Alabaminoides,789,Alabaminoides,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
12,benthic_forams,Alabaminella,423746,Alabaminella,genus,241423,Eponididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN


In [42]:
genus_df.to_csv(genus_path, index=False)

## update genus csv with no higher pbdb ranks
refetch pbdb infor for higher taxa that don't have pbdb data

In [43]:
df = pd.read_csv(genus_path, dtype=str)

temp_df = df[
    df['unranked clade_taxon_id'].isna() &
    df['kingdom_taxon_id'].isna() &
    df['phylum_taxon_id'].isna() &
    df['family_taxon_id'].isna() &
    df['order_taxon_id'].isna() &
    df['class_taxon_id'].isna() &
    df['pbdb_taxon_id'].notna() 
].copy()
log_df(temp_df)


(29, 19)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name
519,dinoflagellates,Comasphaeridium,141162,Comasphaeridium,genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,dinoflagellates,Crassosphaera,264671,Crassosphaera,genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,dinoflagellates,Cyclopsiella,277123,Cyclopsiella,genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
526,dinoflagellates,Cymatiosphaera,170215,Cymatiosphaera,genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,dinoflagellates,Dictyosphaeridium,407647,Dictyosphaeridium,genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
for index, row in temp_df.iterrows():  
        
    time.sleep(0.5)
        
    url =  PBDB_TAXA_ID + str(row['pbdb_taxon_id'])
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            rank = data[0]["taxon_rank"]
            df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
               
            round = 0
            get_parent_taxa(df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)
        else:
            print(name, len(data), ' found')
    else:
        print(name, ' not found')

In [45]:
df.to_csv(genus_path, index=False)

## add genus pbdb to input_file

In [51]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# 7763, 39

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [52]:
genus_df= pd.read_csv(genus_path, dtype=str)

log_df(genus_df)
#1593

(1593, 19)


,taxon_group,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name
0,benthic_forams,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma,774,Adercotryma,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
2,benthic_forams,Alabamina,788,Alabamina,genus,82213,Alabaminidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
3,benthic_forams,Alabaminoides,789,Alabaminoides,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN
4,benthic_forams,Alabaminella,423746,Alabaminella,genus,241423,Eponididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,NaN,NaN


In [54]:
pbdb.add_pbdb_data(df, genus_df, 'genus name')

Bulimina
Parrelloides
Denticulopsis
Denticulopsis
Fenestrella
Rhabdonema
Rhabdonema
Yoshidaia
Corrudinium
Corrudinium
Corrudinium
Corrudinium
Hemiplacophora
Lentinia
Lentinia
Lirella
Nematosphaeropsis
Palaeocystodinium
Piperodinium
Cinctactiniscus
Craniopsis
Craniopsis
Ditripodium
Ebria
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Ebriopsis
Haplohermesinum
Haplohermesinum
Hermesinum
Hermesinum
Hermesinum
Hermesinum
Hermesinum
Parathranium
Parathranium
Parathranium
Parathranium
Parathranium
Parebriopsis
Podamphoropsis
Pseudammodochium
Thranium
Acanthoica
Acanthoica
Acanthoica
Angulolithina
Apertapetra
Aspidorhabdus
Calciopilleus
Cepekiella
Conococcolithus
Conococcolithus
Conococcolithus
Corannulus
Corannulus
Corisphaera
Coronocyclus
Craspedolithus
Craspedolithus
Crassapontosphaera
Crenalithus
Crenalithus
Crenalithus
Crenalithus
Crenalithus
Crenalithus
Cyclococcolithina
Cyclococcolithina
Cyclococcolithin

In [55]:
log_df(df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [56]:
df.to_csv(input_file, index=False)

## create higher taxa csv 

In [80]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


check for for taxon that has both higher taxa and genus name 

In [81]:
df[df['Any taxon above genus'].notna() & df['genus name'].notna()]

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
4265,ostracods,Cytherelloidea sp.,Cytherelloidea indet.,NaN,NaN,indet.,Cytherelloidea,NaN,Cytherelloidea,NaN,...,24710,Platycopida,22826,Ostracoda,18891,Arthropoda,325038,Animalia,NaN,NaN
5965,pollen,Tubiflores sp.,Tubiflorae indet.,NaN,NaN,indet.,Tubiflorae,NaN,Tubiflores,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
higher_df =  pbdb.create_higher_taxa_df(df)

log_df(higher_df)
# 75

(75, 15)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
114,benthic_forams,"""Astronion charlottensis""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,benthic_forams,Miliolidae,81704,Miliolidae,family,81704,Miliolidae,256604,Miliolida,428719,Tubothalamea,288974,Foraminifera,212476,Rhizaria
1555,benthic_forams,Foraminifera,288974,Foraminifera,phylum,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1829,diatoms,Chrysophyta,69586,Chrysophyta,unranked clade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2090,diatoms,"""Dimerogramma elegans""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
higher_df[higher_df.duplicated(subset=['taxon_group', 'Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name


In [77]:
higher_df[higher_df.duplicated(subset=['Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
5577,pollen,Amaranthaceae,55831,Amaryllidaceae,family,55831,Amaryllidaceae,54968,Asparagales,NaN,NaN,54885,Angiospermae,54311,Plantae


In [86]:
pbdb.fetch_pdbd_data(higher_df, 'Any taxon above genus')

"Astronion charlottensis"  not found
"Dimerogramma elegans"  not found
"Diogramma sp."  not found
Globigerinida  not found
5650 Crotonoideae  not found
5750 "Tripodiscinos clavipes"  not found


In [87]:
higher_df.drop_duplicates(inplace=True)

log_df(higher_df)


(75, 15)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
114,benthic_forams,"""Astronion charlottensis""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,benthic_forams,Miliolidae,81704,Miliolidae,family,81704,Miliolidae,256604,Miliolida,428719,Tubothalamea,288974,Foraminifera,212476,Rhizaria
1555,benthic_forams,Foraminifera,288974,Foraminifera,phylum,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1829,diatoms,Chrysophyta,69586,Chrysophyta,unranked clade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2090,diatoms,"""Dimerogramma elegans""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
higher_df.columns

Index(['taxon_group', 'Any taxon above genus', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name'],
      dtype='object')

In [89]:
higher_df.to_csv(higher_taxa_pbdb_file, index=False)

## add higher taxa pbdb to input file

In [90]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [91]:
higher_df = pd.read_csv(higher_taxa_pbdb_file, dtype=str)
log_df(higher_df)
# 75

(75, 15)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,benthic_forams,"""Astronion charlottensis""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Miliolidae,81704,Miliolidae,family,81704,Miliolidae,256604,Miliolida,428719,Tubothalamea,288974,Foraminifera,212476,Rhizaria
2,benthic_forams,Foraminifera,288974,Foraminifera,phylum,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,diatoms,Chrysophyta,69586,Chrysophyta,unranked clade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,diatoms,"""Dimerogramma elegans""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
pbdb.add_pbdb_data(df, higher_df, 'Any taxon above genus')

Ampelidaceae
Combretaceae
Gramineae
Guttiferae
Liliaceae
Nymphaeaceae
Rosaceae
Thymelaeaceae
Urticaceae
Sponguridae


In [94]:
log_df(df)

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [95]:
df.to_csv(input_file, index=False)

## create crosswalk file

In [96]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [97]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [98]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
log_df(crosswalk_df)
# 7763
# 7760
# 7759

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7760, 17)
drop duplicates df:  (7759, 17)
(7759, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,


In [99]:
PI_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [100]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [101]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [102]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7760
# 7658

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7760, 25)
drop duplicates df:  (7658, 25)
(7658, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [103]:
taxa_df.to_csv(taxa_file, index=False)

## temp QA

In [104]:
date = '2021-08-05' # (7763, 20)
date = '2021-11-29' # (7763, 20)
date = '2022-02-24' # (7763, 38)
date = '2022-09-12' # (7763, 38)
date = '2022-10-26' # (7763, 39)
# date = '2022-11-14' # (7763, 39)



date = '2022-10-26' # (7763, 39)
file =  RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
df = pd.read_csv(file, dtype=str)
df.fillna('', inplace=True)
log_df(df)

date = '2022-11-15' # (7763, 39)
file2 =  RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
df2 = pd.read_csv(file2, dtype=str)
df2.fillna('', inplace=True)
log_df(df2)

df.columns == df2.columns

(7763, 39)
(7763, 39)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [105]:
for col in df.columns:
    if len(df) != sum(df[col] == df2[col]):
        print(col)

name
Comment
Notes (change to Internal only notes?)
Any taxon above genus
genus name
species name
subspecies modifier


In [106]:
sum(df['taxon_group'] != df2['taxon_group'])

0

In [107]:
len(df)

7763

In [108]:
df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'Corrections to pbdb_taxon_rank',
       'pbdb_taxon_id.1', 'pbdb_taxon_name.1', 'pbdb_taxon_rank.1',
       'Corrections to pbdb_taxon', 'family_taxon_id', 'family_taxon_name',
       'superfamily_taxon_id', 'superfamily_taxon_name', 'order_taxon_id',
       'order_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [109]:
df[df['pbdb_taxon_name'] != df['pbdb_taxon_name.1']]

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
12,benthic_forams,Alabinella wedellensis,Alabaminella weddellensis,JAS: appears to be a misspelling; WoRMS,,,,,Alabaminella,,...,,,,,288974,Foraminifera,212476,Rhizaria,,
15,benthic_forams,Alveolophraginium crassimargo,Alveolophragmium crassimargo,JAS: appears to be a misspelling; WoRMS,,,,,Alveolophragmium,,...,,,,,288974,Foraminifera,212476,Rhizaria,,
95,benthic_forams,Aragonina aragonensis,Aragonia aragonensis,appears to be a misspelling; WoRMS (JAS),,,,,Aragonia,,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,,
96,benthic_forams,Aragonina velascoensis,Aragonia velascoensis,appears to be a misspelling; WoRMS (JAS),,,,,Aragonia,,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,,
112,benthic_forams,Astrohiza granulosa,Astrorhiza granulosa,appears to be a misspelling; WoRMS (JAS),,,,,Astrorhiza,,...,,,,,288974,Foraminifera,212476,Rhizaria,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5806,pollen,Nuphar,Nuphar sp.,,,,,,Nuphar,,...,54894,Nymphaeales,55217,Magnoliopsida,55350,Spermatophyta,54311,Plantae,,
5836,pollen,Piper sp.,,,,,,,Piper,,...,54891,Piperales,55217,Magnoliopsida,55350,Spermatophyta,54311,Plantae,,
5924,pollen,Taxodiad,Taxodiaceae indet.,Latinized name,,indet.,Taxodiaceae,,,,...,82140,Pinales,82141,Pinopsida,82139,Pinophyta,54311,Plantae,,
5963,pollen,Tsuga sp.,,,,,,,Tsuga,,...,82140,Pinales,82141,Pinopsida,82139,Pinophyta,54311,Plantae,,


In [10]:
df[df['pbdb_taxon_id'] != df['pbdb_taxon_id.1']]

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
12,benthic_forams,Alabinella wedellensis,Alabaminella weddellensis,JAS: appears to be a misspelling; WoRMS,NaN,NaN,NaN,NaN,Alabaminella,NaN,...,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN
15,benthic_forams,Alveolophraginium crassimargo,Alveolophragmium crassimargo,JAS: appears to be a misspelling; WoRMS,NaN,NaN,NaN,NaN,Alveolophragmium,NaN,...,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN
95,benthic_forams,Aragonina aragonensis,Aragonia aragonensis,appears to be a misspelling; WoRMS (JAS),NaN,NaN,NaN,NaN,Aragonia,NaN,...,279579.0,Rotaliida,428504.0,Globothalamea,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN
96,benthic_forams,Aragonina velascoensis,Aragonia velascoensis,appears to be a misspelling; WoRMS (JAS),NaN,NaN,NaN,NaN,Aragonia,NaN,...,279579.0,Rotaliida,428504.0,Globothalamea,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN
112,benthic_forams,Astrohiza granulosa,Astrorhiza granulosa,appears to be a misspelling; WoRMS (JAS),NaN,NaN,NaN,NaN,Astrorhiza,NaN,...,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7747,silicoflagellates,Pseudomicromarsupium lacrimosa,NaN,NaN,"in PBDB, so it will be classified correctly",NaN,NaN,NaN,Pseudomicromarsupium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7753,silicoflagellates,Septamesocena apiculata,NaN,NaN,"in PBDB, so it will be classified correctly",NaN,NaN,NaN,Septamesocena,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7754,silicoflagellates,Septamesocena quadrangula (q),? Septamesocena quadrangula,NaN,NaN,NaN,NaN,?,Septamesocena,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7755,silicoflagellates,Septamesocena sp.,NaN,NaN,NaN,NaN,NaN,NaN,Septamesocena,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
